# 1.29c: Fimbulwinter Duration Analysis

**Question:** How long does the *global freeze* last?

## Definition

**Fimbulwinter** (global freeze): All dead tokens frozen simultaneously.
- W_dead[t+1] == W_dead[t] (bitwise identical, every token)
- Zero displacement for 100% of dead tokens

**Goal:** Find all maximal runs where this condition holds.

Is Fimbulwinter permanent, or does the universe thaw occasionally when a token experiences a non-orthogonal gradient?

## Parameters

In [1]:
# Data path
DATA_PATH = "../tensors/Thimble/thimble_5.safetensors"

print("✓ Parameters set")

✓ Parameters set


## Imports

In [2]:
import torch
import numpy as np
from safetensors.torch import load_file

print("✓ Imports complete")

✓ Imports complete


## Load Data

In [3]:
print("Loading Thimble 5 data...\n")

data = load_file(DATA_PATH)

# Extract W matrix and masks
W_bf16 = data['W']  # (3001, 10000, 64) in bfloat16
dead_mask = data['dead_mask'].bool()

# Extract dead tokens only
W_bf16_dead = W_bf16[:, dead_mask, :]  # (3001, n_dead, 64)

n_steps = W_bf16_dead.shape[0] - 1
n_dead = W_bf16_dead.shape[1]

print(f"W_dead shape: {tuple(W_bf16_dead.shape)}")
print(f"Total steps: {n_steps}")
print(f"Dead tokens: {n_dead}")
print(f"\n✓ Data loaded")

Loading Thimble 5 data...

W_dead shape: (3001, 3699, 64)
Total steps: 3000
Dead tokens: 3699

✓ Data loaded


## Detect Global Freeze

For each timestep t, check if **all** dead tokens are frozen:
- W_dead[t+1] == W_dead[t] (element-wise, bfloat16)

In [4]:
print("Detecting global freeze at each timestep...\n")

# For each timestep, check if ALL tokens are frozen
global_freeze = torch.zeros(n_steps, dtype=torch.bool)

for t in range(n_steps):
    # Check if W[t+1] == W[t] for all dead tokens (bitwise)
    identical = (W_bf16_dead[t+1] == W_bf16_dead[t]).all()
    global_freeze[t] = identical
    
    if t % 500 == 0:
        print(f"t={t}: global freeze = {identical}")

print(f"\n✓ Global freeze detection complete")

Detecting global freeze at each timestep...

t=0: global freeze = False
t=500: global freeze = False
t=1000: global freeze = False
t=1500: global freeze = False
t=2000: global freeze = False
t=2500: global freeze = True

✓ Global freeze detection complete


## Find Maximal Runs

Identify all contiguous intervals where global freeze holds.

In [5]:
print("="*80)
print("FIMBULWINTER RUNS (Global Freeze Intervals)")
print("="*80 + "\n")

# Find runs of True values
runs = []
in_run = False
run_start = None

for t in range(n_steps):
    if global_freeze[t] and not in_run:
        # Start new run
        run_start = t
        in_run = True
    elif not global_freeze[t] and in_run:
        # End current run
        runs.append((run_start, t - 1))
        in_run = False

# Handle case where run extends to end
if in_run:
    runs.append((run_start, n_steps - 1))

print(f"Total Fimbulwinter runs: {len(runs)}\n")

if len(runs) == 0:
    print("No global freeze detected—tokens never all froze simultaneously.")
else:
    print("Runs (start → end, duration):")
    for i, (start, end) in enumerate(runs, 1):
        duration = end - start + 1
        print(f"  Run {i}: t={start} → t={end} (duration: {duration} steps)")
    
    # Statistics
    durations = [end - start + 1 for start, end in runs]
    max_duration = max(durations)
    max_run = runs[durations.index(max_duration)]
    total_frozen_steps = sum(durations)
    
    print(f"\nStatistics:")
    print(f"  Longest run: t={max_run[0]} → t={max_run[1]} ({max_duration} steps)")
    print(f"  Total frozen steps: {total_frozen_steps} / {n_steps} ({100*total_frozen_steps/n_steps:.1f}%)")
    print(f"  Mean run duration: {np.mean(durations):.1f} steps")
    print(f"  Median run duration: {np.median(durations):.1f} steps")

print("\n" + "="*80 + "\n")

FIMBULWINTER RUNS (Global Freeze Intervals)

Total Fimbulwinter runs: 55

Runs (start → end, duration):
  Run 1: t=1143 → t=1145 (duration: 3 steps)
  Run 2: t=1383 → t=1384 (duration: 2 steps)
  Run 3: t=1632 → t=1633 (duration: 2 steps)
  Run 4: t=1639 → t=1640 (duration: 2 steps)
  Run 5: t=1700 → t=1701 (duration: 2 steps)
  Run 6: t=1736 → t=1738 (duration: 3 steps)
  Run 7: t=1747 → t=1751 (duration: 5 steps)
  Run 8: t=1823 → t=1824 (duration: 2 steps)
  Run 9: t=1908 → t=1908 (duration: 1 steps)
  Run 10: t=1917 → t=1923 (duration: 7 steps)
  Run 11: t=1929 → t=1933 (duration: 5 steps)
  Run 12: t=2011 → t=2013 (duration: 3 steps)
  Run 13: t=2030 → t=2030 (duration: 1 steps)
  Run 14: t=2050 → t=2053 (duration: 4 steps)
  Run 15: t=2097 → t=2097 (duration: 1 steps)
  Run 16: t=2100 → t=2110 (duration: 11 steps)
  Run 17: t=2118 → t=2122 (duration: 5 steps)
  Run 18: t=2127 → t=2128 (duration: 2 steps)
  Run 19: t=2141 → t=2147 (duration: 7 steps)
  Run 20: t=2156 → t=2159 (dur

## Late Training Analysis (t=2000–3000)

Focus on the region where we saw 100% frozen in 1.29b.

In [6]:
print("="*80)
print("LATE TRAINING GLOBAL FREEZE (t=2000–3000)")
print("="*80 + "\n")

late_start = 2000
late_frozen = global_freeze[late_start:]

# Runs in late training
late_runs = [(s, e) for s, e in runs if s >= late_start]

print(f"Global freeze percentage in t=2000–3000: {100*late_frozen.float().mean():.1f}%")
print(f"Number of runs: {len(late_runs)}\n")

if len(late_runs) > 0:
    print("Late training runs:")
    for i, (start, end) in enumerate(late_runs, 1):
        duration = end - start + 1
        print(f"  Run {i}: t={start} → t={end} (duration: {duration} steps)")
    
    # Check if final run extends to t=3000
    if late_runs[-1][1] == n_steps - 1:
        print(f"\n✓ Final run extends to t={n_steps} (end of training)")
        print("  Hypothesis: Fimbulwinter is permanent (or at least stable beyond t=3000).")
    else:
        print(f"\n✓ Last thaw occurred at t={late_runs[-1][1] + 1}")
        print("  Hypothesis: Occasional quantum perturbations persist even late in training.")
else:
    print("No global freeze in late training—tokens continue hopping individually.")

print("\n" + "="*80)

LATE TRAINING GLOBAL FREEZE (t=2000–3000)

Global freeze percentage in t=2000–3000: 64.0%
Number of runs: 44

Late training runs:
  Run 1: t=2011 → t=2013 (duration: 3 steps)
  Run 2: t=2030 → t=2030 (duration: 1 steps)
  Run 3: t=2050 → t=2053 (duration: 4 steps)
  Run 4: t=2097 → t=2097 (duration: 1 steps)
  Run 5: t=2100 → t=2110 (duration: 11 steps)
  Run 6: t=2118 → t=2122 (duration: 5 steps)
  Run 7: t=2127 → t=2128 (duration: 2 steps)
  Run 8: t=2141 → t=2147 (duration: 7 steps)
  Run 9: t=2156 → t=2159 (duration: 4 steps)
  Run 10: t=2161 → t=2164 (duration: 4 steps)
  Run 11: t=2169 → t=2177 (duration: 9 steps)
  Run 12: t=2198 → t=2201 (duration: 4 steps)
  Run 13: t=2215 → t=2216 (duration: 2 steps)
  Run 14: t=2219 → t=2219 (duration: 1 steps)
  Run 15: t=2221 → t=2221 (duration: 1 steps)
  Run 16: t=2223 → t=2224 (duration: 2 steps)
  Run 17: t=2227 → t=2234 (duration: 8 steps)
  Run 18: t=2238 → t=2239 (duration: 2 steps)
  Run 19: t=2259 → t=2260 (duration: 2 steps)
  Ru

## Summary

In [7]:
print("\n" + "="*80)
print("FIMBULWINTER DURATION SUMMARY")
print("="*80 + "\n")

total_global_freeze = global_freeze.sum().item()
total_thaw = (~global_freeze).sum().item()

print(f"Global freeze: {total_global_freeze} / {n_steps} steps ({100*total_global_freeze/n_steps:.1f}%)")
print(f"Thaw: {total_thaw} / {n_steps} steps ({100*total_thaw/n_steps:.1f}%)")
print()

if len(runs) > 0:
    max_duration = max(end - start + 1 for start, end in runs)
    print(f"Longest continuous Fimbulwinter: {max_duration} steps")
    
    # Check pattern
    if len(runs) == 1 and runs[0][1] == n_steps - 1:
        print("\n✓ PERMANENT FIMBULWINTER")
        print(f"  System froze at t={runs[0][0]} and never thawed.")
    elif total_global_freeze / n_steps > 0.5:
        print("\n✓ MOSTLY FROZEN with occasional thaws")
        print(f"  {len(runs)} freeze/thaw cycles detected.")
    else:
        print("\n✓ INTERMITTENT FREEZING")
        print(f"  System alternates between freeze and thaw ({len(runs)} cycles).")
else:
    print("\n✓ NO GLOBAL FREEZE")
    print("  At least one token always moving at every timestep.")

print("\n" + "="*80)
print("✓ Fimbulwinter duration analysis complete.")
print("="*80)


FIMBULWINTER DURATION SUMMARY

Global freeze: 674 / 3000 steps (22.5%)
Thaw: 2326 / 3000 steps (77.5%)

Longest continuous Fimbulwinter: 82 steps

✓ INTERMITTENT FREEZING
  System alternates between freeze and thaw (55 cycles).

✓ Fimbulwinter duration analysis complete.
